## <6-3 학습 마친 모델을 실전 투입하기>  

학습을 마친 개체명 인식 모델을 인퍼런스하는 과정을 실습해 보겠습니다.

### 개체명 인식 웹 서비스 만들기  
---
이번 실습에서는 개체명 인식 웹 서비스를 만들어 보겠습니다.  
문장을 토큰화한 뒤 모델 입력값으로 만들고, 이를 모델에 입력해서 문장 내 각 토큰이 특정 개체명 태그일 확률값을 계산한 후 약간의 후처리 과정을 거쳐 응답하는 웹 서비스입니다.

<center><그림 6-5 개체명 인식 웹 서비스></center>

<p align="center"><img src="https://i.imgur.com/I4lGm3J.jpg">   


<center>출처 : ratsgo's NLPBOOK</center>

<**1단계**> **코랩 노트북 초기화하기**     


이전 실습과 마찬가지로 코랩에 접속한 후, `[내 드라이브에 복사]`와 런타임 유형에서 하드웨어 가속을 `[None]`로 선택합니다. 

<**2단계**> **환경 설정하기** 

In [1]:
#의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 350 kB/s 
     |████████████████████████████████| 2.8 MB 21.3 MB/s 
     |████████████████████████████████| 57 kB 3.7 MB/s 
     |████████████████████████████████| 806 kB 35.5 MB/s 
     |████████████████████████████████| 829 kB 49.5 MB/s 
     |████████████████████████████████| 136 kB 45.2 MB/s 
     |████████████████████████████████| 636 kB 48.4 MB/s 
     |████████████████████████████████| 398 kB 35.3 MB/s 
     |████████████████████████████████| 3.3 MB 32.6 MB/s 
     |████████████████████████████████| 895 kB 43.6 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 1.1 MB 57.6 MB/s 
     |████████████████████████████████| 96 kB 5.8 MB/s 
     |████████████████████████████████| 94 kB 2.6 MB/s 
     |████████████████████████████████| 271 kB 56.9 MB/s 
     |████████████████████████████████| 144 kB 62.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

앞 절에서 학습한 모델의 체크포인트는 구글 드라이브에 저장해 두었으므로 다음 코드를 실행해 코랩 노트북과 자신의 구글 드라이브를 연동합니다.

In [2]:
#구글 드라이브 연동
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


이어서 다음 코드를 실행하면 각종 인퍼런스 설정을 수행합니다.  
`pretrained_model_name`과 `max_seq`, `downstream_model_dir` 모두 <6-2>절에서 적용한 그대로 입력해야 합니다.

In [3]:
#인퍼런스 설정
from ratsnlp.nlpbook.ner import NERDeployArguments
args = NERDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/epoch=1-val_loss=0.20-v1.ckpt
downstream_model_labelmap_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/label_map.txt


<**3단계**> **토크나이저 및 모델 불러오기**    


다음 코드를 실행하면 토크나이저를 초기화할 수 있습니다.

In [4]:
#토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

다음 코드를 실행하면 앞 절에서 파인튜닝한 모델의 체크포인트를 읽어들입니다.

In [5]:
#체크포인트 로드
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

그리고 다음 코드를 수행하면 파인튜닝 떄 사용한 `pretrained_model_name`에 해당하는 모델의 설정값들을 읽어들일 수 있습니다.  
이어서 코드를 계속 실행하면 해당 설정값대로 BERT 모델을 초기화합니다.

In [6]:
#BERT 설정 로드
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

In [7]:
#BERT 모델 초기화
from transformers import BertForTokenClassification
model = BertForTokenClassification(pretrained_model_config)

이어서 이제 밑의 코드를 수행하면 방금 초기화한 BERT 모델에 체크포인트를 주입합니다.  

In [8]:
#체크포인트 주입하기
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

모델이 평가 모드로 전환되게 만듭니다.

In [9]:
#평가 모드로 전환
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

<**4단계**> **모델 출력값 만들고 후처리하기**   


개체명 인식 모델의 출력은 각 토큰이 어떤 개체명 태그에 속하는지 확률입니다. 인퍼런스를 하려면 확률값의 각 요솟값이 어떤 태그에 대응하는지 정보를 알고 있어야 합니다.  
이와 관련해 다음 코드를 실행하면 정수 인덱스를 레이블에 매핑하는 사전(dictionary)을 만듭니다. 

In [10]:
#레이블 맵 작성
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}
for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  id_to_label[idx] = label

이 코드를 실행한 결과(`id_to_label`)는 다음과 같습니다.

In [11]:
id_to_label

{0: '[CLS]',
 1: '[SEP]',
 2: '[PAD]',
 3: '[MASK]',
 4: 'O',
 5: '인명',
 6: '기타 수량표현',
 7: '기타',
 8: '기관명',
 9: '날짜',
 10: '지명',
 11: '통화',
 12: '비율',
 13: '시간',
 14: '기간',
 15: '인명',
 16: '기타 수량표현',
 17: '기타',
 18: '기관명',
 19: '날짜',
 20: '지명',
 21: '통화',
 22: '비율',
 23: '시간',
 24: '기간'}

다음 코드는 인퍼런스 과정을 정의한 함수입니다. 문장을 입력받아 토큰화를 수행한 뒤, `input_ids`, `atteintion_mask`, `token_type_ids`를 만듭니다. 이를 입력값을 파이토치 텐서 자료형으로 변환한 뒤 모델에 입력합니다.

In [12]:
#인퍼런스
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        probs = outputs.logits[0].softmax(dim=1)
        top_probs, preds = torch.topk(probs, dim=1, k=1)
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        predicted_tags = [id_to_label[pred.item()] for pred in preds]
        result = []
        for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
            if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
                token_result = {
                    "token": token,
                    "predicted_tag": predicted_tag,
                    "top_prob": str(round(top_prob[0].item(), 4)),
                }
                result.append(token_result)
    return {
        "sentence": sentence,
        "result": result,
    }

모델 출력값(`outputs.logit`)은 토큰 각각에 대해 반환되며 소프트맥스 함수 적용 이전의 로짓입니다. 여기에 소프트맥스 함수를 써서 해당 토큰이 특정 개체명 태그일 확률 형태로 바꿉니다.  
마지막으로 모델 출력을 약간 후처리하여 예측 확률의 최댓값과 해당 태그가 출력되도록 `pred`값을 만듭니다.

<**5단계**> **웹 서비스 시작하기**    

앞 단계에서 정의한 인퍼런스 함수 `inference_fn()`을 가지고 다음 코드를 실행하면 웹 서비스를 시작할 수 있습니다.



In [14]:
!mkdir /root/.ngrok2 && echo "authtoken: 279c6rgTRIX2kOe4VIaoCJXy0u8_2fAuAWUJwnY1Q22wwoqra" > /root/.ngrok2/ngrok.yml

In [15]:
#웹 서비스 시작
from ratsnlp.nlpbook.ner import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.ner.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://dc4f-35-245-248-95.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
